In [129]:
#pyvizenv environment needed
#commented out libraries & APIs we aren't currently using - kp

import requests
# initial imports
import os
import pandas as pd
#import panel as pn
#import plotly.express as px
#import plotly.graph_objects as go
#import holoviews as hv
#from pathlib import Path
import datetime
from dotenv import load_dotenv
import json


load_dotenv()
rapid_api_key = os.getenv("RAPID_API_KEY")

In [105]:
#import hvplot.pandas
#import matplotlib.pyplot as plt

In [106]:
# Read the Mapbox API key
#load_dotenv()
#mapbox_token = os.getenv("MAPBOX")
#px.set_mapbox_access_token(mapbox_token)

In [107]:
#alpaca_token = os.getenv("ALPACA_API_KEY")

In [71]:
# outputs textbox to key in stock symbol and uses api to get historical stock data for that stock
# use code to loop through list of tickers and output daily close data for x amount of time
# updated ticker list to be in same format - kp
# changed order in loop to print ticker symbol at top - kp
# updated url to make api data in json format, changed return to dump with indents - kp

tickers = ["BTC-USD", "DJIA", "XAU/USD", "DXY", "NDX", "SPX", "WTI"]

def data_grab(ticker):
    """
    Takes in arg "ticker" as a str object,
    and returns daily stock information for that
    symbol/ticker.
    
    date/open/high/low/close/volume/adjclose
    """
    
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
    url = url + "?format=json"

    querystring = {"symbol":""}
    querystring['symbol']=ticker
    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    
    response_data = requests.get(url, headers=headers, params=querystring)
    daily_data = response_data.json()
    daily_prices = daily_data['prices']
    
    for daily_info in daily_prices:
        try:
            print(daily_info['close'])
        except:
            print("N/A")
        
    return print("Done")

    #return(print(json.dumps(daily_data, indent=4)))

#for ticker in tickers:
        #print(f"\n {ticker} BEGINNING")
        #data_grab(ticker)
        #print(f"\n {ticker} END")

In [142]:
#Initial Variables
closing_price_list = []
closing_date_list = []

#API Information
url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
url = url + "?format=json"
querystring = {"symbol":""}
querystring['symbol']="TSLA"
headers = {
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }

#Response Information
response_data = requests.get(url, headers=headers, params=querystring)
daily_data = response_data.json()
daily_prices = daily_data['prices']

#Loop to grab closing price
for daily_info in daily_prices:
    try:
        closing_date_list.append(datetime.datetime.fromtimestamp(daily_info['date']).strftime('%Y-%m-%d'))
        closing_price_list.append(daily_info['close'])
    except:
        print("N/A")
        
clean_date_list = list(dict.fromkeys(closing_date_list))

d = {'Dates':clean_date_list, "Closing Prices":closing_price_list}
df = pd.DataFrame(data=d)
        
df

N/A


,Dates,Closing Prices
0,2020-09-18,442.149994
1,2020-09-17,423.429993
2,2020-09-16,441.760010
3,2020-09-15,449.760010
4,2020-09-14,419.619995
5,2020-09-11,372.720001
6,2020-09-10,371.339996
7,2020-09-09,366.279999
8,2020-09-08,330.209991
9,2020-09-04,418.320007


In [ ]:
# NEXT STEPS
# create dataframes with api data